In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

## READING FILES

In [2]:
date_respuesta = '2022-01-31'
date_respuesta = pd.to_datetime(date_respuesta)

In [3]:
# disp = pd.read_excel(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\2020\10. OCTUBRE\Dispersión-20201014.xlsx'
#                      ,sheet_name='Swap del día',converters={'CLABE':str,'report_id':str})
disp = pd.read_csv(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2022\01. JANUARY\SWAP 2022-01-31.csv'
                     ,dtype={'bank_account_number':object,'report_id':object})
rs1 = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\2022\01. ENERO\Respuesta-Swap 2022-01-31.csv',skiprows=11
                     ,dtype={'Descripcion':object,'Referencia Numerica':object,'Rastreo CEP':object})
ae1 = pd.read_csv(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2022\01. JANUARY\SwapDOWNLOAD 2022-01-31.csv'
                  ,names=['Nombre','Blank','amount','CLABE','report_id'],dtype={'CLABE':object,'report_id':object},skiprows=[0])
cep1 = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\2022\01. ENERO\CEP 2022-01-31.txt'
                ,names=['Fecha','Clave_rastreo','Banco_emisor','Banco_receptor','Cuenta','Monto']
                   ,dtype={'Fecha':object,'Clave_rastreo':object,'Banco_emisor':object,'Banco_receptor':object,'Cuenta':object})
bin1 = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\Catalogo_Bancos.csv'
                   ,dtype={'BIN':object,'Clave de la institución':object,'Nombre de la institución':object})


# BIN

In [4]:
bin1.head()

,BIN,Clave de la institución,Nombre de la institución
0,138,40138,ABC CAPITAL
1,102,40102,ACCENDO BANCO
2,133,40133,ACTINVER
3,062,40062,AFIRME
4,638,90638,AKALA


In [5]:
bin1.shape

(88, 3)

# ARCHIVO CEPS

In [6]:
cep1['Linea_CEP'] = cep1['Fecha']+','+cep1['Clave_rastreo']+','+cep1['Banco_emisor']+','+cep1['Banco_receptor']+','+cep1['Cuenta']+','+cep1['Monto'].astype(str)

In [7]:
cep1.head()

,Fecha,Clave_rastreo,Banco_emisor,Banco_receptor,Cuenta,Monto,Linea_CEP
2022-01-31,2022013140014 BET0000435796410,40014,40062,062580008321210372,AFIRME,0.95,"2022013140014 BET0000435796410,40014,40062,062..."
2022-01-31,2022013140014 BET0000435797080,40014,40137,137930104154721242,BANCOPPEL,4.79,"2022013140014 BET0000435797080,40014,40137,137..."
2022-01-31,#N/D,40014,40014,014566606225834532,SANTANDER,"9,965.70","#N/D,40014,40014,014566606225834532,SANTANDER,..."
2022-01-31,2022013140014 BET0000435797720,40014,40021,021050040583305567,HSBC,105.41,"2022013140014 BET0000435797720,40014,40021,021..."
2022-01-31,2022013140014 BET0000435798260,40014,40127,127180016882159005,AZTECA,824.15,"2022013140014 BET0000435798260,40014,40127,127..."


In [8]:
cep1.shape

(58031, 7)

# ARCHIVO ENVIADO A SWAP

In [9]:
ae1.head()

,Nombre,Blank,amount,CLABE,report_id
0,Gabriela Cruz Aragon,nan,8.00,012180011642955876,CLIP-VBRNQFSJV
1,Juan Jose Abarca Barreto,nan,"1,830.23",012040011224470291,CLIP-2Z4R2SP2X
2,Victor Hugo Flores Munoz,nan,"1,437.36",002180902707725277,CLIP-QYWYS4NWR
3,JPA SOLUCIONES AEREAS S A DE C V,nan,"30,320.63",012691001161237406,CLIP-5WR3TNVFT
4,JESSON FERNANDO MARTINEZ ROBLES,nan,59.41,072060010927954789,CLIP-822PPZ8LT


In [10]:
ae1.shape

(58031, 5)

In [11]:
# ae1['report_id'] = ae1['report_id'].str[5:].replace(' ', '',regex=True)
ae1['CLABE'] = ae1['CLABE'].replace(' ', '',regex=True)
ae1['report_id'] = ae1['report_id'].replace(' ', '',regex=True)
ae1['BIN'] = ae1['CLABE'].str[:3].replace(' ', '',regex=True)

In [12]:
ae_final = ae1.merge(bin1[['BIN','Clave de la institución','Nombre de la institución']] ,on='BIN', how='left')
ae_final.head()

,Nombre,Blank,amount,CLABE,report_id,BIN,Clave de la institución,Nombre de la institución
0,Gabriela Cruz Aragon,nan,8.00,012180011642955876,CLIP-VBRNQFSJV,012,40012,BBVA BANCOMER
1,Juan Jose Abarca Barreto,nan,"1,830.23",012040011224470291,CLIP-2Z4R2SP2X,012,40012,BBVA BANCOMER
2,Victor Hugo Flores Munoz,nan,"1,437.36",002180902707725277,CLIP-QYWYS4NWR,002,40002,BANAMEX
3,JPA SOLUCIONES AEREAS S A DE C V,nan,"30,320.63",012691001161237406,CLIP-5WR3TNVFT,012,40012,BBVA BANCOMER
4,JESSON FERNANDO MARTINEZ ROBLES,nan,59.41,072060010927954789,CLIP-822PPZ8LT,072,40072,BANORTE/IXE


In [13]:
ae_final.shape

(58031, 8)

# ARCHIVO RESPUESTA

In [14]:
rs1['Descripcion'] = rs1['Descripcion'].str.strip()

In [15]:
rs1['Date_respuesta'] = date_respuesta
rs1['Metodo de dispersion'] = 'SWAP'
rs1.rename(columns ={'Descripcion':'report_id'}, inplace=True)
rs1['Rastreo CEP'].fillna('SIN CEP',inplace=True)
rs1.head()

,Referencia Numerica,Rastreo CEP,Nombre,report_id,Cuenta,Banco,Importe,Comision,Estatus,Mensaje,Date_respuesta,Metodo de dispersion
0,1100001,2022013140014 BET0000435796410,MARLEM BERENICE MERCADO LARA,R8KFKSC66,'062580008321210372,AFIRME,0.95,2.15,paid,NaN,2022-01-31,SWAP
1,1100002,2022013140014 BET0000435797080,VICENTE VAZQUEZ CALDERON,VYNNTX2CW,'137930104154721242,BANCOPPEL,4.79,2.15,paid,NaN,2022-01-31,SWAP
2,1100003,#N/D,THOMASCURTBARTER,ZHYSPMBHM,'014566606225834532,SANTANDER,"9,965.70",2.15,paid,NaN,2022-01-31,SWAP
3,1100004,2022013140014 BET0000435797720,JUAN PABLO MARTNEZ VILLARREAL,HTBYCJKVQ,'021050040583305567,HSBC,105.41,2.15,paid,NaN,2022-01-31,SWAP
4,1100005,2022013140014 BET0000435798260,KEYLA DANAE MUNOZ HURTADO,48DV5YX9S,'127180016882159005,AZTECA,824.15,2.15,paid,NaN,2022-01-31,SWAP


In [16]:
rs1.shape

(58031, 12)

In [17]:
rs1['Rastreo CEP'].str[:4].unique()

array(['2022', '#N/D', '0', 'CLIP', '17EB'], dtype=object)

In [18]:
rs1.loc[rs1['Rastreo CEP'].str.contains("'085"), "Linea_rastreo"] = rs1['Rastreo CEP'].str[1:]
rs1.loc[~rs1['Rastreo CEP'].str.contains("'085"), "Linea_rastreo"] = rs1['Rastreo CEP']


In [19]:
rs1.loc[rs1['Rastreo CEP'].str[:4] == "'085", "Banco_emisor"] = 'BANAMEX'
rs1.loc[rs1['Rastreo CEP'].str[:4] == 'CLIP', "Banco_emisor"] = 'STP'
rs1.loc[rs1['Rastreo CEP'].str[:4] == '2021', "Banco_emisor"] = 'SANTANDER'
rs1.loc[rs1['Rastreo CEP'].str[:4] == '2022', "Banco_emisor"] = 'SANTANDER'
rs1.loc[rs1['Rastreo CEP'].str[:4] == '#N/D', "Banco_emisor"] = 'SANTANDER'
rs1.loc[rs1['Rastreo CEP'].str[:3] == '178', "Banco_emisor"] = 'BANAMEX'
rs1.loc[rs1['Rastreo CEP'].str[:2] == '17', "Banco_emisor"] = 'BANAMEX'
rs1.loc[rs1['Rastreo CEP'].str[:4] == '1.78', "Banco_emisor"] = 'BANAMEX'
rs1.loc[rs1['Rastreo CEP'].str[:4] == '1.79', "Banco_emisor"] = 'BANAMEX'
rs1.loc[rs1['Rastreo CEP'].str[:4] == '0', "Banco_emisor"] = 'SANTANDER - REFERENCIA NUMERICA'
rs1.loc[rs1['Rastreo CEP'].str[:4] == '#¡REF!', "Banco_emisor"] = 'CEP PENDIENTE DE ENVIO'


In [20]:
rs1.loc[rs1['Banco_emisor'] == 'BANAMEX', "Emisor"] = '40002'
rs1.loc[rs1['Banco_emisor'] == 'STP', "Emisor"] = '90646'
rs1.loc[rs1['Banco_emisor'] == 'SANTANDER', "Emisor"] = '40014' 


In [21]:
rs1['Cuenta'].str[0:1].unique()

array(["'"], dtype=object)

In [22]:
rs1['BIN'] = rs1['Cuenta'].str[1:4]
rs1['CLABE'] = rs1['Cuenta'].str[1:]


In [23]:
rs1.head()

,Referencia Numerica,Rastreo CEP,Nombre,report_id,Cuenta,Banco,Importe,Comision,Estatus,Mensaje,Date_respuesta,Metodo de dispersion,Linea_rastreo,Banco_emisor,Emisor,BIN,CLABE
0,1100001,2022013140014 BET0000435796410,MARLEM BERENICE MERCADO LARA,R8KFKSC66,'062580008321210372,AFIRME,0.95,2.15,paid,NaN,2022-01-31,SWAP,2022013140014 BET0000435796410,SANTANDER,40014,062,062580008321210372
1,1100002,2022013140014 BET0000435797080,VICENTE VAZQUEZ CALDERON,VYNNTX2CW,'137930104154721242,BANCOPPEL,4.79,2.15,paid,NaN,2022-01-31,SWAP,2022013140014 BET0000435797080,SANTANDER,40014,137,137930104154721242
2,1100003,#N/D,THOMASCURTBARTER,ZHYSPMBHM,'014566606225834532,SANTANDER,"9,965.70",2.15,paid,NaN,2022-01-31,SWAP,#N/D,SANTANDER,40014,014,014566606225834532
3,1100004,2022013140014 BET0000435797720,JUAN PABLO MARTNEZ VILLARREAL,HTBYCJKVQ,'021050040583305567,HSBC,105.41,2.15,paid,NaN,2022-01-31,SWAP,2022013140014 BET0000435797720,SANTANDER,40014,021,021050040583305567
4,1100005,2022013140014 BET0000435798260,KEYLA DANAE MUNOZ HURTADO,48DV5YX9S,'127180016882159005,AZTECA,824.15,2.15,paid,NaN,2022-01-31,SWAP,2022013140014 BET0000435798260,SANTANDER,40014,127,127180016882159005


In [24]:
rs1.shape

(58031, 17)

In [25]:
rs2 = rs1.merge(bin1[['BIN','Clave de la institución','Nombre de la institución']] ,on='BIN', how='left')
rs2['LINEA_CEP'] = rs2['Date_respuesta'].astype(str).replace(' ', '',regex=True)+','+rs2['Linea_rastreo'].astype(str)+','+rs2['Emisor'].astype(str).replace(' ', '',regex=True)+','+rs2['Clave de la institución'].astype(str).replace(' ', '',regex=True)+','+rs2['CLABE'].astype(str).replace(' ', '',regex=True)+','+rs2['Importe'].astype(str).replace(' ', '',regex=True)
rs2.head()

,Referencia Numerica,Rastreo CEP,Nombre,report_id,Cuenta,Banco,Importe,Comision,Estatus,Mensaje,Date_respuesta,Metodo de dispersion,Linea_rastreo,Banco_emisor,Emisor,BIN,CLABE,Clave de la institución,Nombre de la institución,LINEA_CEP
0,1100001,2022013140014 BET0000435796410,MARLEM BERENICE MERCADO LARA,R8KFKSC66,'062580008321210372,AFIRME,0.95,2.15,paid,NaN,2022-01-31,SWAP,2022013140014 BET0000435796410,SANTANDER,40014,062,062580008321210372,40062,AFIRME,"2022-01-31,2022013140014 BET0000435796410,4001..."
1,1100002,2022013140014 BET0000435797080,VICENTE VAZQUEZ CALDERON,VYNNTX2CW,'137930104154721242,BANCOPPEL,4.79,2.15,paid,NaN,2022-01-31,SWAP,2022013140014 BET0000435797080,SANTANDER,40014,137,137930104154721242,40137,BANCOPPEL,"2022-01-31,2022013140014 BET0000435797080,4001..."
2,1100003,#N/D,THOMASCURTBARTER,ZHYSPMBHM,'014566606225834532,SANTANDER,"9,965.70",2.15,paid,NaN,2022-01-31,SWAP,#N/D,SANTANDER,40014,014,014566606225834532,40014,SANTANDER,"2022-01-31,#N/D,40014,40014,014566606225834532..."
3,1100004,2022013140014 BET0000435797720,JUAN PABLO MARTNEZ VILLARREAL,HTBYCJKVQ,'021050040583305567,HSBC,105.41,2.15,paid,NaN,2022-01-31,SWAP,2022013140014 BET0000435797720,SANTANDER,40014,021,021050040583305567,40021,HSBC,"2022-01-31,2022013140014 BET0000435797720,4001..."
4,1100005,2022013140014 BET0000435798260,KEYLA DANAE MUNOZ HURTADO,48DV5YX9S,'127180016882159005,AZTECA,824.15,2.15,paid,NaN,2022-01-31,SWAP,2022013140014 BET0000435798260,SANTANDER,40014,127,127180016882159005,40127,AZTECA,"2022-01-31,2022013140014 BET0000435798260,4001..."


In [26]:
rs2.shape

(58031, 20)

# ARCHIVO DISPERSION

In [27]:
disp['report_id'] = disp['report_id'].str.strip()
disp['report_date_local'] = pd.to_datetime(disp['report_date_local'])

In [28]:
disp.loc[disp['report_date_local'] == date_respuesta, 'tipo'] = 'Pago del día'
disp.loc[disp['report_date_local'] != date_respuesta, 'tipo'] = 'Reprogramación'
disp.drop(['bank_name','bank_account_number','status','swap','payment_date_local'],axis=1,inplace=True)
disp.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_account_owner_name,batch_created_date,tipo
0,0000d758-84b4-4cb0-a165-4f0d3c44864d,MAHARISHI PANTJALI SA DE CV,maharishiclip@gmail.com,75HNWQ7W8,2022-01-31,"66,652.53",MAHARISHI PANTJALI S.A. DE C.V.,2022-01-31 09:03:18,Pago del día
1,0000e953-6b2c-44d0-a798-9dac1abf24ea,Super Kandy,yanellysegovia18@gmail.com,4YYNJCD2T,2022-01-31,"5,039.84",Ana Yanelly Puente Gutierrez,2022-01-31 09:03:18,Pago del día
2,00023bd5-6149-4ef9-95be-edb61d22588d,LAS PEKES,alebarradas1@hotmail.com,DBM9TZH9J,2022-01-31,43.12,Virginia Alejandra Barradas Prez,2022-01-31 09:03:18,Pago del día
3,000255c1-03db-49c8-993d-2ce223408b0a,La Mision,micaelaguerrero3531@gmail.com,DKLFCG7B7,2022-01-31,61.81,Micaela Guerrero,2022-01-31 09:03:18,Pago del día
4,0003c12e-9130-4af9-b4be-dba67bfeab51,Nakama Sushi,nakama.sushi.pfp@gmail.com,XQ6WWFZNM,2022-01-31,182.07,Chtistian David Guzman Renteria,2022-01-31 09:03:18,Pago del día


In [29]:
disp_2 = disp.merge(rs2[['report_id','Estatus','Nombre de la institución','CLABE','Banco_emisor','LINEA_CEP','Referencia Numerica']] ,on='report_id', how='left')
disp_2.sort_values(by=['Estatus'],ascending=True,inplace=True)
disp_2.sort_values(by=['Estatus'],ascending=True,inplace=True)
disp_2['Estatus'].fillna('not_found',inplace=True)
disp_2['Nombre de la institución'].fillna('not_found',inplace=True)
disp_2['CLABE'].fillna('not_found',inplace=True)
disp_2['Banco_emisor'].fillna('not_found',inplace=True)
disp_2['LINEA_CEP'].fillna('not_found',inplace=True)
disp_2['Referencia Numerica'].fillna('not_found',inplace=True)
disp_2.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_account_owner_name,batch_created_date,tipo,Estatus,Nombre de la institución,CLABE,Banco_emisor,LINEA_CEP,Referencia Numerica
19127,54cf4446-37a7-4c3f-9b48-f83c27a37a65,Gambrinus,Marthalopez@hotmail.com,YH25MZHY9,2022-01-31,"4,464.44",Aniela Jazette Hernandez Serrano,2022-01-31 09:06:12,Pago del día,failed,BBVA BANCOMER,012180015732411297,SANTANDER - REFERENCIA NUMERICA,"2022-01-31,0,nan,40012,012180015732411297,4464.44",1237774
1283,05c1d600-b4de-4e5c-b232-62129077f39d,Sevilla,sevilla-omar@hotmail.com,3BWQC3SYT,2022-01-31,"5,006.80",Jose Omar Sevilla Jurez,2022-01-31 09:03:35,Pago del día,failed,BBVA BANCOMER,012180015173153062,SANTANDER - REFERENCIA NUMERICA,"2022-01-31,0,nan,40012,012180015173153062,5006.8",1209216
26510,759467a3-6aab-42f1-be99-a926cd0fb201,Miscelanea Citlali,jmejia693@yahoo.com,J95VL2PRD,2022-01-31,92.95,Juan Pedro Mejia Aldaba,2022-01-31 09:07:34,Pago del día,failed,BANAMEX,002060701462490252,SANTANDER - REFERENCIA NUMERICA,"2022-01-31,0,nan,40002,002060701462490252,92.95",1248282
4917,15d09508-4545-4e12-8708-14c25bcc70e9,Los Inmortales Food,torrescaro97@gmail.com,7P6J8FVDM,2022-01-31,2.00,Carolina Torres,2022-01-31 09:04:18,Pago del día,failed,SANTANDER,014078605789866166,SANTANDER,"2022-01-31,#N/D,40014,40014,014078605789866166...",1212223
46001,cadf723b-2f1b-490d-b4ff-153e279f885b,Barbacoa Osorio,enrique301269@hotmail.com,K49X6JRH7,2022-01-31,471.58,Mariana Melissa Osorio Robles,2022-01-31 09:10:21,Pago del día,failed,BANAMEX,002180930141974096,SANTANDER - REFERENCIA NUMERICA,"2022-01-31,0,nan,40002,002180930141974096,471.58",1227118


In [30]:
disp_2.loc[disp_2['LINEA_CEP'].str.contains('#N/D'), 'LINEA_CEP'] = 'COMPROBANTE MISMO BANCO SANTANDER'
disp_2['LINEA_CEP'] = np.where(disp_2['Banco_emisor'] == 'SANTANDER - REFERENCIA NUMERICA', disp_2['Referencia Numerica'], disp_2['LINEA_CEP'])
disp_2.loc[disp_2['Estatus'] == 'failed', 'LINEA_CEP'] = ''
disp_2.loc[disp_2['Estatus'] == 'failed', 'Banco_emisor'] = ''
disp_2.loc[disp_2['LINEA_CEP'] == 'CEP PENDIENTE DE ENVIO', 'Banco_emisor'] = ''

In [31]:
for col in disp_2.columns: 
    print(col) 

merchant_id
merchant_name
user_email
report_id
report_date_local
amount
bank_account_owner_name
batch_created_date
tipo
Estatus
Nombre de la institución
CLABE
Banco_emisor
LINEA_CEP
Referencia Numerica


In [32]:
disp_final = disp_2[['merchant_id','merchant_name','user_email','report_id','report_date_local','amount','tipo'
                    ,'Nombre de la institución','bank_account_owner_name','CLABE','Estatus','LINEA_CEP','Banco_emisor']]
disp_final.rename(columns ={'Nombre de la institución':'bank','Estatus':'status','amount':'total_due'}, inplace=True)
disp_final.sort_values(by=['status'], ascending=True, inplace=True)
disp_final.head()

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,merchant_id,merchant_name,user_email,report_id,report_date_local,total_due,tipo,bank,bank_account_owner_name,CLABE,status,LINEA_CEP,Banco_emisor
19127,54cf4446-37a7-4c3f-9b48-f83c27a37a65,Gambrinus,Marthalopez@hotmail.com,YH25MZHY9,2022-01-31,"4,464.44",Pago del día,BBVA BANCOMER,Aniela Jazette Hernandez Serrano,012180015732411297,failed,,
38540,aaae533a-67e7-4670-8c4d-8418be43719e,MiniT,mario-30g.c@hotmail.com,WXNMM2MNC,2022-01-31,716.78,Pago del día,BBVA BANCOMER,Mario Antonio Garca Castaeda,012180015301826967,failed,,
44360,c38d53d0-f5b1-485b-845f-c6df69ef1a51,Veterinaria Martinez Luna,veterinarialunamtz@gmail.com,T5GWFFNDK,2022-01-31,753.90,Pago del día,BBVA BANCOMER,GLORIA PORTALES SAUCEDO,012180015802685746,failed,,
8261,24bfb6be-2dd1-48a0-a1b0-ab2aa6eaf603,Abarrotes El Cuadro,lola_030890@hotmail.com,QWWNKPVFL,2022-01-31,27.31,Pago del día,COMPARTAMOS,Dolores Adriana Perez Ramirez,130560001439231600,failed,,
28211,7cf0e2a3-abe6-4301-b65c-bfff73577957,Productos Naturales,sandraortega596@gmail.com,4PP4SSHCJ,2022-01-31,"4,982.85",Pago del día,SCOTIABANK,Sandra Basauri Ortega,044180256007504174,failed,,


In [33]:
disp_final.shape

(58031, 13)

In [34]:
duplicates_disp_final = disp_final[disp_final.duplicated(['report_id'],keep=False)]
duplicates_disp_final.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,total_due,tipo,bank,bank_account_owner_name,CLABE,status,LINEA_CEP,Banco_emisor


In [35]:
duplicates_disp_final.shape

(0, 13)

# ERROR

In [36]:
error = disp_final[(disp_final['status'] == 'failed')]
error.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,total_due,tipo,bank,bank_account_owner_name,CLABE,status,LINEA_CEP,Banco_emisor
19127,54cf4446-37a7-4c3f-9b48-f83c27a37a65,Gambrinus,Marthalopez@hotmail.com,YH25MZHY9,2022-01-31,"4,464.44",Pago del día,BBVA BANCOMER,Aniela Jazette Hernandez Serrano,012180015732411297,failed,,
38540,aaae533a-67e7-4670-8c4d-8418be43719e,MiniT,mario-30g.c@hotmail.com,WXNMM2MNC,2022-01-31,716.78,Pago del día,BBVA BANCOMER,Mario Antonio Garca Castaeda,012180015301826967,failed,,
44360,c38d53d0-f5b1-485b-845f-c6df69ef1a51,Veterinaria Martinez Luna,veterinarialunamtz@gmail.com,T5GWFFNDK,2022-01-31,753.90,Pago del día,BBVA BANCOMER,GLORIA PORTALES SAUCEDO,012180015802685746,failed,,
8261,24bfb6be-2dd1-48a0-a1b0-ab2aa6eaf603,Abarrotes El Cuadro,lola_030890@hotmail.com,QWWNKPVFL,2022-01-31,27.31,Pago del día,COMPARTAMOS,Dolores Adriana Perez Ramirez,130560001439231600,failed,,
28211,7cf0e2a3-abe6-4301-b65c-bfff73577957,Productos Naturales,sandraortega596@gmail.com,4PP4SSHCJ,2022-01-31,"4,982.85",Pago del día,SCOTIABANK,Sandra Basauri Ortega,044180256007504174,failed,,


In [37]:
error.shape

(329, 13)

In [38]:
error.drop(['bank_account_owner_name','CLABE','status','LINEA_CEP','Banco_emisor'],axis=1,inplace=True)

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [39]:
error.loc[error['bank'] == 'SANTANDER', 'Check'] = 'SPEI DEVUELTO MISMO BANCO'
error.loc[error['bank'] != 'SANTANDER', 'Check'] = 'SPEI DEVUELTO'

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [40]:
error_final = error.merge(rs2[['report_id','Mensaje']] ,on='report_id', how='left')
error_final['Mensaje'].fillna('NOT FOUND',inplace=True)
error_final.head()


,merchant_id,merchant_name,user_email,report_id,report_date_local,total_due,tipo,bank,Check,Mensaje
0,54cf4446-37a7-4c3f-9b48-f83c27a37a65,Gambrinus,Marthalopez@hotmail.com,YH25MZHY9,2022-01-31,"4,464.44",Pago del día,BBVA BANCOMER,SPEI DEVUELTO,Excede el limite de saldo autorizado de la cue...
1,aaae533a-67e7-4670-8c4d-8418be43719e,MiniT,mario-30g.c@hotmail.com,WXNMM2MNC,2022-01-31,716.78,Pago del día,BBVA BANCOMER,SPEI DEVUELTO,Excede el limite de saldo autorizado de la cue...
2,c38d53d0-f5b1-485b-845f-c6df69ef1a51,Veterinaria Martinez Luna,veterinarialunamtz@gmail.com,T5GWFFNDK,2022-01-31,753.90,Pago del día,BBVA BANCOMER,SPEI DEVUELTO,Excede el limite de saldo autorizado de la cue...
3,24bfb6be-2dd1-48a0-a1b0-ab2aa6eaf603,Abarrotes El Cuadro,lola_030890@hotmail.com,QWWNKPVFL,2022-01-31,27.31,Pago del día,COMPARTAMOS,SPEI DEVUELTO,Tipo de operacion erronea
4,7cf0e2a3-abe6-4301-b65c-bfff73577957,Productos Naturales,sandraortega596@gmail.com,4PP4SSHCJ,2022-01-31,"4,982.85",Pago del día,SCOTIABANK,SPEI DEVUELTO,Excede el limite de saldo autorizado de la cue...


In [41]:
error_final.shape

(329, 10)

# ARCHIVOS FINALES

### ARCHIVO DISPERSION

In [42]:
writer = pd.ExcelWriter('DISPERSION SWAP.xlsx', engine='xlsxwriter', options={'encoding':'utf-8'})
disp_final.to_excel(writer, sheet_name='SWAP DEL DIA',index=False)
error_final.to_excel(writer, sheet_name='ERROR',index=False)
writer.save()

### ARCHIVO RESPUESTA

In [43]:
rs2.head()

,Referencia Numerica,Rastreo CEP,Nombre,report_id,Cuenta,Banco,Importe,Comision,Estatus,Mensaje,Date_respuesta,Metodo de dispersion,Linea_rastreo,Banco_emisor,Emisor,BIN,CLABE,Clave de la institución,Nombre de la institución,LINEA_CEP
0,1100001,2022013140014 BET0000435796410,MARLEM BERENICE MERCADO LARA,R8KFKSC66,'062580008321210372,AFIRME,0.95,2.15,paid,NaN,2022-01-31,SWAP,2022013140014 BET0000435796410,SANTANDER,40014,062,062580008321210372,40062,AFIRME,"2022-01-31,2022013140014 BET0000435796410,4001..."
1,1100002,2022013140014 BET0000435797080,VICENTE VAZQUEZ CALDERON,VYNNTX2CW,'137930104154721242,BANCOPPEL,4.79,2.15,paid,NaN,2022-01-31,SWAP,2022013140014 BET0000435797080,SANTANDER,40014,137,137930104154721242,40137,BANCOPPEL,"2022-01-31,2022013140014 BET0000435797080,4001..."
2,1100003,#N/D,THOMASCURTBARTER,ZHYSPMBHM,'014566606225834532,SANTANDER,"9,965.70",2.15,paid,NaN,2022-01-31,SWAP,#N/D,SANTANDER,40014,014,014566606225834532,40014,SANTANDER,"2022-01-31,#N/D,40014,40014,014566606225834532..."
3,1100004,2022013140014 BET0000435797720,JUAN PABLO MARTNEZ VILLARREAL,HTBYCJKVQ,'021050040583305567,HSBC,105.41,2.15,paid,NaN,2022-01-31,SWAP,2022013140014 BET0000435797720,SANTANDER,40014,021,021050040583305567,40021,HSBC,"2022-01-31,2022013140014 BET0000435797720,4001..."
4,1100005,2022013140014 BET0000435798260,KEYLA DANAE MUNOZ HURTADO,48DV5YX9S,'127180016882159005,AZTECA,824.15,2.15,paid,NaN,2022-01-31,SWAP,2022013140014 BET0000435798260,SANTANDER,40014,127,127180016882159005,40127,AZTECA,"2022-01-31,2022013140014 BET0000435798260,4001..."


In [44]:
rs_final = rs2[['Referencia Numerica','Rastreo CEP','Nombre','report_id','Cuenta','Banco','Importe','Comision','Estatus'
                ,'Mensaje','Date_respuesta','Metodo de dispersion']]
rs_final.sort_values(by=['Estatus'], ascending=True, inplace=True)
rs_final.head()

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Referencia Numerica,Rastreo CEP,Nombre,report_id,Cuenta,Banco,Importe,Comision,Estatus,Mensaje,Date_respuesta,Metodo de dispersion
41347,1242260,0,LAURA LUCIA VAZQUEZ AGUILAR,NRRHP883Q,'044680256036746872,SCOTIABANK,"1,685.16",2.15,failed,Falta informacion mandatoria para completar el...,2022-01-31,SWAP
23054,1232969,0,JUAN PABLO LEIJA HERNANDEZ,R5CQ4Z8B2,'012180015620973436,BBVA BANCOMER,622.86,2.15,failed,Excede el limite de saldo autorizado de la cue...,2022-01-31,SWAP
30792,1229989,0,JOSE ARMANDO CORTES BURGOS,VQ7LS8WT9,'021180040426294100,HSBC,383.30,2.15,failed,Cuenta bloqueada,2022-01-31,SWAP
25626,1232408,0,ANDREA VALERIA MARTINEZ CASTILLO,C38WGS977,'012180015682075084,BBVA BANCOMER,727.30,2.15,failed,Excede el limite de saldo autorizado de la cue...,2022-01-31,SWAP
15112,1228247,0,HUGO GUTIERREZ,9GBS9Z4FZ,'012180028006385444,BBVA BANCOMER,382.33,2.15,failed,Excede el limite de saldo autorizado de la cue...,2022-01-31,SWAP


In [45]:
rs_final.shape

(58031, 12)

In [46]:
rs_final.to_csv(r'G:\My Drive\Settlement\Respuestas Swap\2022\01. Enero 2022\Respuesta 2022-01-31.csv',index=False)